In [2]:
import mne
import sys
import os
import importlib


from braindecode.preprocessing.preprocess import preprocess, Preprocessor, zscore
from braindecode.datasets import (create_from_mne_raw, create_from_mne_epochs)
from braindecode.preprocessing.windowers import create_windows_from_events
from braindecode.datasets.sleep_physionet import SleepPhysionet
from braindecode.datasets import BaseDataset, BaseConcatDataset, WindowsDataset

from plot import Plot

In [ ]:

print('LOADING BCI DATA')

''' ANNOTATIONS
T0 corresponds to rest
T1 corresponds to onset of motion (real or imagined) of
    the left fist (in runs 3, 4, 7, 8, 11, and 12)
    both fists (in runs 5, 6, 9, 10, 13, and 14)
T2 corresponds to onset of motion (real or imagined) of
    the right fist (in runs 3, 4, 7, 8, 11, and 12)
    both feet (in runs 5, 6, 9, 10, 13, and 14)
'''

subject_size = [1,10]
subjects = range(subject_size[0], subject_size[1]) # max 110
event_codes = [
    1, 2, # eyes open, eyes closed (baselines)
    3, 4, 5,
    6, 7, 8, 9, 
    10, 11, 12, 13, 14
]

physionet_paths, descriptions = [], []

for subject_id in subjects:
    physionet_paths += [mne.datasets.eegbci.load_data(subject_id, event_codes, update_path=False)]
    descriptions += [{"event_code": code, "subject": subject_id} for code in event_codes]


In [2]:
# raw_set = [
#     '/home/maligan/mne_data/physionet-sleep-data/SC4012E0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4451F0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4441E0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4431E0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4421E0-PSG.edf',
# ]

# space_bambi directory
data_dir = './data/SPACE_BAMBI_2channels/'
raw_set = [
    f'{data_dir}SPACE.S175.yyyymmdd.EORASD_raw.fif',
    f'{data_dir}SPACE.S170.yyyymmdd.EORASD_raw.fif'
]

print(raw_set)

# load into raw array
raws = [mne.io.read_raw_fif(x, preload=True) for x in raw_set]

['./data/SPACE_BAMBI_2channels/SPACE.S175.yyyymmdd.EORASD_raw.fif', './data/SPACE_BAMBI_2channels/SPACE.S170.yyyymmdd.EORASD_raw.fif']
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S175.yyyymmdd.EORASD_raw.fif...
    Range : 0 ... 628735 =      0.000 ...   307.000 secs
Ready.
Reading 0 ... 628735  =      0.000 ...   307.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S170.yyyymmdd.EORASD_raw.fif...
    Range : 0 ... 616447 =      0.000 ...   301.000 secs
Ready.
Reading 0 ... 616447  =      0.000 ...   301.000 secs...


In [35]:
# space_bambi directory
data_dir = './data/SPACE_BAMBI_2channels/'

raws = []

print(f'{len(os.listdir(data_dir))} files found')
for i, path in enumerate(os.listdir(data_dir)):
    if i == 5:
        break
    full_path = os.path.join(data_dir, path)
    raws.append(mne.io.read_raw_fif(full_path, preload=True))

340 files found
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S27.yyyymmdd.ECRASD_raw.fif...
    Range : 0 ... 368639 =      0.000 ...   180.000 secs
Ready.
Reading 0 ... 368639  =      0.000 ...   180.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/BAMBI.S620.yyyymmdd.EORASD1_raw.fif...
    Range : 0 ... 618495 =      0.000 ...   302.000 secs
Ready.
Reading 0 ... 618495  =      0.000 ...   302.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S50.yyyymmdd.EORASD_raw.fif...
    Range : 0 ... 378879 =      0.000 ...   185.000 secs
Ready.
Reading 0 ... 378879  =      0.000 ...   185.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S101.yyyymmdd.EORHE_raw.fif...
    Range : 0 ... 622591 =      0.000 ...   304.000 secs
Ready.
Reading 0 ... 622591  =      0.000 ...   304.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S94.yyyymmdd.EORHE_raw.fif...
    Range : 0 ... 731135 =      0.000 ...   357.000 secs
Re

In [36]:
len(raws)

5

In [26]:
raws[1].annotations

<Annotations | 29 segments: artifact (29)>

In [4]:
### preprocess
# resample to 160Hz
# high pass filtering of 30Hz

sfreq = 160
high_cut_hz = 30
n_jobs = 2

for raw in raws:
    mne.io.Raw.resample(raw, sfreq, n_jobs=n_jobs) # resample all files
    mne.io.Raw.filter(raw, l_freq=None, h_freq=high_cut_hz, n_jobs=n_jobs) # high-pass filter

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 71 samples (0.444 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 71 samples (0.444 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished


In [5]:
base_datasets = [BaseDataset(raw) for raw in raws]
base_concat_datasets = BaseConcatDataset(base_datasets)

In [6]:
base_concat_datasets

In [7]:
window_size_samples = 100

windows_dataset = create_from_mne_raw(
    raws,
    trial_start_offset_samples=0,
    trial_stop_offset_samples=0,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples,
    drop_last_window=True,
    # descriptions=descriptions,
    accepted_bads_ratio=0.5,
    drop_bad_windows=True,
    # mapping=mapping,
    # preload=True,
    on_missing='ignore'
)


Used Annotations descriptions: ['artifact']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
112 matching events found
No baseline correction applied
0 projection items activated
Loading data for 112 events and 100 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['artifact']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
120 matching events found
No baseline correction applied
0 projection items activated
Loading data for 120 events and 100 original time points ...
0 bad epochs dropped


In [8]:
type(windows_dataset)

braindecode.datasets.base.BaseConcatDataset

In [9]:
# z-score normalization

preprocess(windows_dataset, Preprocessor(zscore))


Loading data for 112 events and 100 original time points ...
Loading data for 120 events and 100 original time points ...


/home/maligan/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function zscore is deprecated; will be removed in 0.7.0. Use sklearn.preprocessing.scale instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: 'WindowsDataset' object has no attribute '__annotations__'